This notebook gives a simple example of how to use the ShaderMaterial to write custom shaders from the Python side. For further information about the shaders, consult the three.js docs.

In [ ]:
from pythreejs import *
import ipywidgets
from IPython.display import display

In [ ]:
ShaderMaterial?

In [ ]:
vertex_shader = """
uniform float time;
uniform vec2 resolution;

void main() {
    vec3 pos = vec3(position.x + time * resolution.x, position.y + time * resolution.y, position.z);
    gl_Position = projectionMatrix * modelViewMatrix * vec4( pos, 1.0 );
}
"""

In [ ]:
fragment_shader = """
void main() {
    #ifdef OVERRIDE_COLOR
    gl_FragColor = vec4(0, 0.5, 0, 1.0);
    #else
    gl_FragColor = vec4(0.5, 0, 0, 1.0);
    #endif
}
"""

In [ ]:
material = ShaderMaterial(
    uniforms=dict(
        time=dict(value=0.0),
        resolution=dict(value=(1, 1)),
        **UniformsLib['lights']
    ),
    defines=dict(
        OVERRIDE_COLOR=1,
    ),
    vertexShader=vertex_shader,
    fragmentShader=fragment_shader,
    lights=True,
)

In [ ]:
material

In [ ]:
material.defines = dict()
material.needsUpdate = True

In [ ]:
def update_time(time):
    uniforms = dict(**material.uniforms)
    uniforms.update(time=dict(value=time))
    material.uniforms = uniforms
    material.needsUpdate = True

In [ ]:
update_time(-15)